# Simulating Constitutive Processes of semantic change within heterogeneous populations of speakers

In [1]:
# basic imports
import os
import torch
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

# project code imports
from mod.one_hot_agent import *
from mod.plot import *
from mod.network import *

##### Hyper parameters

In [2]:
model_version = 'dy-1hot'

In [3]:
turns = 300
no_agents = 2
no_connections = 1
add_vocab_in = 50
semantic_features = 3
starting_observations = 5
starting_uncertainty = .2
words_per_semantic_feature = 5
new_environment_prob = .25
enforce_word_feature_mapping = True

In [4]:
model_path = os.path.join('html',model_version)
if not os.path.exists(model_path):
    os.mkdir(model_path)

## Simple nteraction across repeated turns in a randomly generated environment

In [5]:
env_name = 'Randomly-Generated-Environment'

In [6]:
starting_env = torch.distributions.MultivariateNormal(torch.randn(size=(1,semantic_features)), covariance_matrix=torch.eye(semantic_features) * .2)

In [7]:
net = social_network(
    k_agents=no_agents,
    vocab_size=words_per_semantic_feature,
    connections_per_agent=no_connections,
    semantic_dimensions=semantic_features,
    starting_observations=starting_observations,
    starting_uncertainty=starting_uncertainty,
    enforcing=enforce_word_feature_mapping
)

In [8]:
fig = heatmap(
    net.similarity_heatmap(),
    x_labels=['agent_'+str(i+1) for i in range(len(net.agents))],
    y_labels=['agent_'+str(i+1) for i in range(len(net.agents))]
)
fig.show()

In [9]:
vocab_dif, H_dif = [], []

In [10]:
for rd in tqdm(range(turns)):
    env = starting_env.sample()

    net.interaction(env, 3)
    vocab_dif += [{'agent':i + 1, 'interaction': rd, 'Δ P(w|m)':val} for i,val in enumerate(net.similarity_heatmap().mean(dim=-1).tolist())]
    H_dif += [{'agent':i + 1, 'interaction': rd, 'Δ P(w|m)':val} for i,val in enumerate(net.network_entropy().tolist())]

vocab_dif = pd.DataFrame(vocab_dif)
H_dif = pd.DataFrame(H_dif)

  0%|          | 0/300 [00:00<?, ?it/s]

In [11]:
fig = heatmap(
    net.similarity_heatmap(),
    x_labels=['agent_'+str(i+1) for i in range(len(net.agents))],
    y_labels=['agent_'+str(i+1) for i in range(len(net.agents))]
)
fig.show()

In [12]:
fig = multi_agent_update_plot_from_df(vocab_dif)
fig.update_layout(
    # title='Dyadic interaction in a random environment',
    yaxis_title='Δ P(w|m)',
    xaxis_title='interactions'
)
fig.show()

In [13]:
fig.write_html(
    os.path.join(model_path,env_name+'.html')
)

In [14]:
fig = multi_agent_update_plot_from_df(H_dif)
fig.update_layout(
    # title='Dyadic interaction in a random environment',
    yaxis_title='H(vocab)',
    xaxis_title='interactions'
)
fig.show()

In [15]:
fig.write_html(
    os.path.join(model_path,env_name+'-H.html')
)

## Random environment and introduction of new terms

In [16]:
env_name = 'Randomly-Generated-Environment-and-Introducing-Novel-Terms'

In [17]:
starting_env = torch.distributions.MultivariateNormal(torch.randn(size=(1,semantic_features)), covariance_matrix=torch.eye(semantic_features) * .2)

In [18]:
net = social_network(
    k_agents=no_agents,
    vocab_size=words_per_semantic_feature,
    connections_per_agent=no_connections,
    semantic_dimensions=semantic_features,
    starting_observations=starting_observations,
    starting_uncertainty=starting_uncertainty,
    enforcing=enforce_word_feature_mapping
)

In [19]:
fig = heatmap(
    net.similarity_heatmap(),
    x_labels=['agent_'+str(i+1) for i in range(len(net.agents))],
    y_labels=['agent_'+str(i+1) for i in range(len(net.agents))]
)
fig.show()

In [20]:
vocab_dif, H_dif = [], []

In [21]:
for rd in tqdm(range(turns)):
    env = starting_env.sample()

    new_vocab_round = ((rd % add_vocab_in) == 0) & (rd != 0)
    net.interaction(env, 3, new_vocab_round)

    vocab_dif += [{'agent':i + 1, 'interaction': rd, 'Δ P(w|m)':val} for i,val in enumerate(net.similarity_heatmap().mean(dim=-1).tolist())]
    H_dif += [{'agent':i + 1, 'interaction': rd, 'Δ P(w|m)':val} for i,val in enumerate(net.network_entropy().tolist())]

vocab_dif = pd.DataFrame(vocab_dif)
H_dif = pd.DataFrame(H_dif)

  0%|          | 0/300 [00:00<?, ?it/s]

In [22]:
fig = heatmap(
    net.similarity_heatmap(),
    x_labels=['agent_'+str(i+1) for i in range(len(net.agents))],
    y_labels=['agent_'+str(i+1) for i in range(len(net.agents))]
)
fig.show()

In [23]:
fig = multi_agent_update_plot_from_df(vocab_dif)
fig.update_layout(
    # title='Dyadic interaction in a random environment',
    yaxis_title='Δ P(w|m)',
    xaxis_title='interactions'
)
fig.show()

In [24]:
fig.write_html(
    os.path.join(model_path,env_name+'.html')
)

In [25]:
fig = multi_agent_update_plot_from_df(H_dif)
fig.update_layout(
    # title='Dyadic interaction in a random environment',
    yaxis_title='H(vocab)',
    xaxis_title='interactions'
)
fig.show()

In [26]:
fig.write_html(
    os.path.join(model_path,env_name+'-H.html')
)

## Changing environment

In [27]:
env_name = 'Suddenly-Changing-Environment'

In [28]:
starting_env = torch.distributions.MultivariateNormal(torch.randn(size=(1,semantic_features)), covariance_matrix=torch.eye(semantic_features) * .2)

In [29]:
net = social_network(
    k_agents=no_agents,
    vocab_size=words_per_semantic_feature,
    connections_per_agent=no_connections,
    semantic_dimensions=semantic_features,
    starting_observations=starting_observations,
    starting_uncertainty=starting_uncertainty,
    enforcing=enforce_word_feature_mapping
)

In [30]:
fig = heatmap(
    net.similarity_heatmap(),
    x_labels=['agent_'+str(i+1) for i in range(len(net.agents))],
    y_labels=['agent_'+str(i+1) for i in range(len(net.agents))]
)
fig.show()

In [31]:
vocab_dif, H_dif = [], []

In [32]:
for rd in tqdm(range(turns)):

    new_env_prob = torch.rand(size=(1,))
    if new_env_prob > new_environment_prob:
        starting_env = torch.distributions.MultivariateNormal(torch.randn(size=(1,semantic_features)), covariance_matrix=torch.eye(semantic_features) * .2)

    env = starting_env.sample()

    new_vocab_round = False
    net.interaction(env, 3, new_vocab_round)

    vocab_dif += [{'agent':i + 1, 'interaction': rd, 'Δ P(w|m)':val} for i,val in enumerate(net.similarity_heatmap().mean(dim=-1).tolist())]
    H_dif += [{'agent':i + 1, 'interaction': rd, 'Δ P(w|m)':val} for i,val in enumerate(net.network_entropy().tolist())]

vocab_dif = pd.DataFrame(vocab_dif)
H_dif = pd.DataFrame(H_dif)

  0%|          | 0/300 [00:00<?, ?it/s]

In [33]:
fig = heatmap(
    net.similarity_heatmap(),
    x_labels=['agent_'+str(i+1) for i in range(len(net.agents))],
    y_labels=['agent_'+str(i+1) for i in range(len(net.agents))]
)
fig.show()

In [34]:
fig = multi_agent_update_plot_from_df(vocab_dif)
fig.update_layout(
    # title='Dyadic interaction in a random environment',
    yaxis_title='Δ P(w|m)',
    xaxis_title='interactions'
)
fig.show()

In [35]:
fig.write_html(
    os.path.join(model_path,env_name+'.html')
)

In [36]:
fig = multi_agent_update_plot_from_df(H_dif)
fig.update_layout(
    # title='Dyadic interaction in a random environment',
    yaxis_title='H(vocab)',
    xaxis_title='interactions'
)
fig.show()

In [37]:
fig.write_html(
    os.path.join(model_path,env_name+'-H.html')
)

## Changing environment and introduction of new terms

In [38]:
env_name = 'Suddenly-Changing-Environment-and-Introducing-Novel-Terms'

In [39]:
starting_env = torch.distributions.MultivariateNormal(torch.randn(size=(1,semantic_features)), covariance_matrix=torch.eye(semantic_features) * .2)

In [40]:
net = social_network(
    k_agents=no_agents,
    vocab_size=words_per_semantic_feature,
    connections_per_agent=no_connections,
    semantic_dimensions=semantic_features,
    starting_observations=starting_observations,
    starting_uncertainty=starting_uncertainty,
    enforcing=enforce_word_feature_mapping
)

In [41]:
fig = heatmap(
    net.similarity_heatmap(),
    x_labels=['agent_'+str(i+1) for i in range(len(net.agents))],
    y_labels=['agent_'+str(i+1) for i in range(len(net.agents))]
)
fig.show()

In [42]:
vocab_dif, H_dif = [], []

In [43]:
for rd in tqdm(range(turns)):

    new_env_prob = torch.rand(size=(1,))
    if new_env_prob > new_environment_prob:
        starting_env = torch.distributions.MultivariateNormal(torch.randn(size=(1,semantic_features)), covariance_matrix=torch.eye(semantic_features) * .2)

    env = starting_env.sample()

    new_vocab_round = ((rd % add_vocab_in) == 0) & (rd != 0)
    net.interaction(env, 3, new_vocab_round)

    vocab_dif += [{'agent':i + 1, 'interaction': rd, 'Δ P(w|m)':val} for i,val in enumerate(net.similarity_heatmap().mean(dim=-1).tolist())]
    H_dif += [{'agent':i + 1, 'interaction': rd, 'Δ P(w|m)':val} for i,val in enumerate(net.network_entropy().tolist())]

vocab_dif = pd.DataFrame(vocab_dif)
H_dif = pd.DataFrame(H_dif)

  0%|          | 0/300 [00:00<?, ?it/s]

In [44]:
fig = heatmap(
    net.similarity_heatmap(),
    x_labels=['agent_'+str(i+1) for i in range(len(net.agents))],
    y_labels=['agent_'+str(i+1) for i in range(len(net.agents))]
)
fig.show()

In [45]:
fig = multi_agent_update_plot_from_df(vocab_dif)
fig.update_layout(
    # title='Dyadic interaction in a random environment',
    yaxis_title='Δ P(w|m)',
    xaxis_title='interactions'
)
fig.show()

In [46]:
fig.write_html(
    os.path.join(model_path,env_name+'.html')
)

In [47]:
fig = multi_agent_update_plot_from_df(H_dif)
fig.update_layout(
    # title='Dyadic interaction in a random environment',
    yaxis_title='H(vocab)',
    xaxis_title='interactions'
)
fig.show()

In [48]:
fig.write_html(
    os.path.join(model_path,env_name+'-H.html')
)

## Stochastic environment

In [49]:
env_name = 'Stochastically-Changing-Environment'

In [50]:
starting_env = torch.distributions.MultivariateNormal(torch.randn(size=(1,semantic_features)), covariance_matrix=torch.eye(semantic_features) * .2)

In [51]:
net = social_network(
    k_agents=no_agents,
    vocab_size=words_per_semantic_feature,
    connections_per_agent=no_connections,
    semantic_dimensions=semantic_features,
    starting_observations=starting_observations,
    starting_uncertainty=starting_uncertainty,
    enforcing=enforce_word_feature_mapping
)

In [52]:
fig = heatmap(
    net.similarity_heatmap(),
    x_labels=['agent_'+str(i+1) for i in range(len(net.agents))],
    y_labels=['agent_'+str(i+1) for i in range(len(net.agents))]
)
fig.show()

In [53]:
vocab_dif, H_dif = [], []

In [54]:
for rd in tqdm(range(turns)):

    env = starting_env.sample()
    starting_env.loc = env

    new_vocab_round = False
    net.interaction(env, 3, new_vocab_round)

    vocab_dif += [{'agent':i + 1, 'interaction': rd, 'Δ P(w|m)':val} for i,val in enumerate(net.similarity_heatmap().mean(dim=-1).tolist())]
    H_dif += [{'agent':i + 1, 'interaction': rd, 'Δ P(w|m)':val} for i,val in enumerate(net.network_entropy().tolist())]

vocab_dif = pd.DataFrame(vocab_dif)
H_dif = pd.DataFrame(H_dif)

  0%|          | 0/300 [00:00<?, ?it/s]

In [55]:
fig = heatmap(
    net.similarity_heatmap(),
    x_labels=['agent_'+str(i+1) for i in range(len(net.agents))],
    y_labels=['agent_'+str(i+1) for i in range(len(net.agents))]
)
fig.show()

In [56]:
fig = multi_agent_update_plot_from_df(vocab_dif)
fig.update_layout(
    # title='Dyadic interaction in a random environment',
    yaxis_title='Δ P(w|m)',
    xaxis_title='interactions'
)
fig.show()

In [57]:
fig.write_html(
    os.path.join(model_path,env_name+'.html')
)

In [58]:
fig = multi_agent_update_plot_from_df(H_dif)
fig.update_layout(
    # title='Dyadic interaction in a random environment',
    yaxis_title='H(vocab)',
    xaxis_title='interactions'
)
fig.show()

In [59]:
fig.write_html(
    os.path.join(model_path,env_name+'-H.html')
)

## Stochastic environment and introduction of new terms

In [60]:
env_name = 'Stochastically-Changing-Environment-and-Introducing-Novel-Terms'

In [61]:
starting_env = torch.distributions.MultivariateNormal(torch.randn(size=(1,semantic_features)), covariance_matrix=torch.eye(semantic_features) * .2)

In [62]:
net = social_network(
    k_agents=no_agents,
    vocab_size=words_per_semantic_feature,
    connections_per_agent=no_connections,
    semantic_dimensions=semantic_features,
    starting_observations=starting_observations,
    starting_uncertainty=starting_uncertainty,
    enforcing=enforce_word_feature_mapping
)

In [63]:
fig = heatmap(
    net.similarity_heatmap(),
    x_labels=['agent_'+str(i+1) for i in range(len(net.agents))],
    y_labels=['agent_'+str(i+1) for i in range(len(net.agents))]
)
fig.show()

In [64]:
vocab_dif, H_dif = [], []

In [65]:
for rd in tqdm(range(turns)):

    env = starting_env.sample()
    starting_env.loc = env

    new_vocab_round = ((rd % add_vocab_in) == 0) & (rd != 0)
    net.interaction(env, 3, new_vocab_round)

    vocab_dif += [{'agent':i + 1, 'interaction': rd, 'Δ P(w|m)':val} for i,val in enumerate(net.similarity_heatmap().mean(dim=-1).tolist())]
    H_dif += [{'agent':i + 1, 'interaction': rd, 'Δ P(w|m)':val} for i,val in enumerate(net.network_entropy().tolist())]

vocab_dif = pd.DataFrame(vocab_dif)
H_dif = pd.DataFrame(H_dif)

  0%|          | 0/300 [00:00<?, ?it/s]

In [66]:
fig = heatmap(
    net.similarity_heatmap(),
    x_labels=['agent_'+str(i+1) for i in range(len(net.agents))],
    y_labels=['agent_'+str(i+1) for i in range(len(net.agents))]
)
fig.show()

In [67]:
fig = multi_agent_update_plot_from_df(vocab_dif)
fig.update_layout(
    # title='Dyadic interaction in a random environment',
    yaxis_title='Δ P(w|m)',
    xaxis_title='interactions'
)
fig.show()

In [68]:
fig.write_html(
    os.path.join(model_path,env_name+'.html')
)

In [69]:
fig = multi_agent_update_plot_from_df(H_dif)
fig.update_layout(
    # title='Dyadic interaction in a random environment',
    yaxis_title='H(vocab)',
    xaxis_title='interactions'
)
fig.show()

In [70]:
fig.write_html(
    os.path.join(model_path,env_name+'-H.html')
)

## Sudden and stochastic environment

In [71]:
env_name = 'Suddenly-and-Stochastically-Changing-Environment'

In [72]:
starting_env = torch.distributions.MultivariateNormal(torch.randn(size=(1,semantic_features)), covariance_matrix=torch.eye(semantic_features) * .2)

In [73]:
net = social_network(
    k_agents=no_agents,
    vocab_size=words_per_semantic_feature,
    connections_per_agent=no_connections,
    semantic_dimensions=semantic_features,
    starting_observations=starting_observations,
    starting_uncertainty=starting_uncertainty,
    enforcing=enforce_word_feature_mapping
)

In [74]:
fig = heatmap(
    net.similarity_heatmap(),
    x_labels=['agent_'+str(i+1) for i in range(len(net.agents))],
    y_labels=['agent_'+str(i+1) for i in range(len(net.agents))]
)
fig.show()

In [75]:
vocab_dif, H_dif = [], []

In [76]:
for rd in tqdm(range(turns)):

    new_env_prob = torch.rand(size=(1,))
    if new_env_prob > new_environment_prob:
        starting_env = torch.distributions.MultivariateNormal(torch.randn(size=(1,semantic_features)), covariance_matrix=torch.eye(semantic_features) * .2)

    env = starting_env.sample()
    starting_env.loc = env

    new_vocab_round = False
    net.interaction(env, 3, new_vocab_round)

    vocab_dif += [{'agent':i + 1, 'interaction': rd, 'Δ P(w|m)':val} for i,val in enumerate(net.similarity_heatmap().mean(dim=-1).tolist())]
    H_dif += [{'agent':i + 1, 'interaction': rd, 'Δ P(w|m)':val} for i,val in enumerate(net.network_entropy().tolist())]

vocab_dif = pd.DataFrame(vocab_dif)
H_dif = pd.DataFrame(H_dif)

  0%|          | 0/300 [00:00<?, ?it/s]

In [77]:
fig = heatmap(
    net.similarity_heatmap(),
    x_labels=['agent_'+str(i+1) for i in range(len(net.agents))],
    y_labels=['agent_'+str(i+1) for i in range(len(net.agents))]
)
fig.show()

In [78]:
fig = multi_agent_update_plot_from_df(vocab_dif)
fig.update_layout(
    # title='Dyadic interaction in a random environment',
    yaxis_title='Δ P(w|m)',
    xaxis_title='interactions'
)
fig.show()

In [79]:
fig.write_html(
    os.path.join(model_path,env_name+'.html')
)

In [80]:
fig = multi_agent_update_plot_from_df(H_dif)
fig.update_layout(
    # title='Dyadic interaction in a random environment',
    yaxis_title='H(vocab)',
    xaxis_title='interactions'
)
fig.show()

In [81]:
fig.write_html(
    os.path.join(model_path,env_name+'-H.html')
)

## Sudden and stochastic environment, plus introduction of new terms

In [82]:
env_name = 'Suddenly-and-Stochastically-Changing-Environment-and-Introducing-Novel-Terms'

In [83]:
starting_env = torch.distributions.MultivariateNormal(torch.randn(size=(1,semantic_features)), covariance_matrix=torch.eye(semantic_features) * .2)

In [84]:
net = social_network(
    k_agents=no_agents,
    vocab_size=words_per_semantic_feature,
    connections_per_agent=no_connections,
    semantic_dimensions=semantic_features,
    starting_observations=starting_observations,
    starting_uncertainty=starting_uncertainty,
    enforcing=enforce_word_feature_mapping
)

In [85]:
fig = heatmap(
    net.similarity_heatmap(),
    x_labels=['agent_'+str(i+1) for i in range(len(net.agents))],
    y_labels=['agent_'+str(i+1) for i in range(len(net.agents))]
)
fig.show()

In [86]:
vocab_dif, H_dif = [], []

In [87]:
for rd in tqdm(range(turns)):

    new_env_prob = torch.rand(size=(1,))
    if new_env_prob > new_environment_prob:
        starting_env = torch.distributions.MultivariateNormal(torch.randn(size=(1,semantic_features)), covariance_matrix=torch.eye(semantic_features) * .2)

    env = starting_env.sample()
    starting_env.loc = env

    new_vocab_round = ((rd % add_vocab_in) == 0) & (rd != 0)
    net.interaction(env, 3, new_vocab_round)

    vocab_dif += [{'agent':i + 1, 'interaction': rd, 'Δ P(w|m)':val} for i,val in enumerate(net.similarity_heatmap().mean(dim=-1).tolist())]
    H_dif += [{'agent':i + 1, 'interaction': rd, 'Δ P(w|m)':val} for i,val in enumerate(net.network_entropy().tolist())]

vocab_dif = pd.DataFrame(vocab_dif)
H_dif = pd.DataFrame(H_dif)

  0%|          | 0/300 [00:00<?, ?it/s]

In [88]:
fig = heatmap(
    net.similarity_heatmap(),
    x_labels=['agent_'+str(i+1) for i in range(len(net.agents))],
    y_labels=['agent_'+str(i+1) for i in range(len(net.agents))]
)
fig.show()

In [89]:
fig = multi_agent_update_plot_from_df(vocab_dif)
fig.update_layout(
    # title='Dyadic interaction in a random environment',
    yaxis_title='Δ P(w|m)',
    xaxis_title='interactions'
)
fig.show()

In [90]:
fig.write_html(
    os.path.join(model_path,env_name+'.html')
)

In [91]:
fig = multi_agent_update_plot_from_df(H_dif)
fig.update_layout(
    # title='Dyadic interaction in a random environment',
    yaxis_title='H(vocab)',
    xaxis_title='interactions'
)
fig.show()

In [92]:
fig.write_html(
    os.path.join(model_path,env_name+'-H.html')
)